In [1]:
from __future__ import print_function
import numpy as np
import cv2
from PIL import Image, ImageEnhance, ImageFilter, ImageOps
import glob
import os

from ShapeDetect import ShapeDetect as sd
from ArrowDetect import ArrowDetect as ad
from TextDetect_OPENCV import TextDetectAll as tda
from Diag2Graph_v2 import Diag2Graph as tgv2
import pytesseract
from ParseJSON import ParseJSON as pj

# Comment below line for LINUX - Update below path for WINDOWS
# pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [2]:
def preprocessImage(image_path, resize):
    
    # load the image from disk and then preprocess it
    image = cv2.imread(image_path)
    # add white border in the original image        
    image = cv2.copyMakeBorder(image,10,10,10,10,cv2.BORDER_CONSTANT,value=[255, 255, 255]) 
    if resize == 1:
        newX, newY = image.shape[1]*1.5, image.shape[0]*1.5
    else:
        newX, newY = image.shape[1], image.shape[0]
            
    image_resize = cv2.resize(image,(int(newX),int(newY)))
        
    imgPIL = Image.open(image_path)
    imgPIL = ImageOps.expand(imgPIL, border = 10, fill = 'white')
    imgPIL = imgPIL.resize((int(newX),int(newY)), Image.ANTIALIAS)        
    imgPIL = ImageEnhance.Color(imgPIL)
    imgPIL = imgPIL.enhance(0)
    gray_im = imgPIL.convert('L') 

    gray_imcv = np.array(gray_im, dtype=np.uint8)    
    _, thresh_im = cv2.threshold(gray_imcv, 240, 255, cv2.THRESH_BINARY_INV)    
        
    return image_resize, thresh_im, gray_imcv

In [3]:
imdir_path = "Input"
op_dir = "Output"

In [4]:
print("[INFO] loading images ...")
    
for filename in glob.glob(os.path.join(imdir_path, '*png')):
    print(filename)
    im, thresh_im, gray_imcv = preprocessImage(filename, 0)
    
    parsejson = pj()
    paper_title, paper_file_name, paper_conf, paper_year, fig_caption, fig_text = parsejson.getCaption(filename)
    if not os.path.isdir(os.path.join(op_dir, paper_file_name)):
        os.mkdir(os.path.join(op_dir, paper_file_name))
        os.mkdir(os.path.join(op_dir, paper_file_name, "diag2graph"))
        os.mkdir(os.path.join(op_dir, paper_file_name, "Figures"))

    cv2.imwrite(os.path.join(op_dir, paper_file_name+ "/Figures/" + os.path.basename(filename)), im)
    
    shapedetector = sd()
    component, flow_dir = shapedetector.find_component(filename, op_dir, im, thresh_im, gray_imcv)
               
    textdetector = tda()
    text_list = textdetector.combinedTextDetect(filename, im, component, fig_text)
        
    arrowdetector = ad()            
    line_connect = arrowdetector.detectLines(im, thresh_im, gray_imcv, component, text_list)
    
    graphcreator = tgv2()
    graphcreator.createDiag2Graph(op_dir, filename, im, thresh_im, component, flow_dir, text_list, line_connect, paper_title, paper_file_name, paper_conf, paper_year, fig_caption)
        
        
        

[INFO] loading images ...
Input\fig_1802.06006v2-Figure2-1.png
4
4
Input\fig_1802.06006v2-Figure2-1.png
Input\fig_1807.03039v2-Figure2-1.png
11
11
Input\fig_1807.03039v2-Figure2-1.png
Input\fig_7237-modulating-early-visual-processing-by-language-Figure1-1.png
13
13
Input\fig_7237-modulating-early-visual-processing-by-language-Figure1-1.png
Input\fig_7272-real-time-image-saliency-for-black-box-classifiers-Figure4-1.png
16
16
Input\fig_7272-real-time-image-saliency-for-black-box-classifiers-Figure4-1.png
Input\fig_Alperovich_Light_Field_Intrinsics_CVPR_2018_paper-Figure3-1.png
7
7
Input\fig_Alperovich_Light_Field_Intrinsics_CVPR_2018_paper-Figure3-1.png
